In [0]:
import plotly.offline as py
import plotly.graph_objs as go
import pandas as pd

py.offline.init_notebook_mode(connected=True)

#import data

#datapath = "C:\Users\Dan Steffy\Dropbox\Research\VRPVIS\\"
datapath = "/Users/TomCreo/Desktop/"

df_ArrivalTimes = pd.read_excel(str(datapath+"Assignment30.xlsx"), sheet_name="ArrivalTimes")
df_Locations = pd.read_excel(str(datapath+"Assignment30.xlsx"), sheet_name="Locations")
df_ArrivalLoad = pd.read_excel(str(datapath+"Assignment30.xlsx"), sheet_name="ArrivalLoad")
df_Demand = pd.read_excel(str(datapath+"Assignment30.xlsx"), sheet_name="Demand")
df_Requests = pd.read_csv(str(datapath+"Google_Campus_Data_217_Shifted.csv"))

#to store requestID associated with each stop
VehRequestID = []

#to store latitude associated with each stop
VehLat = []

#to store longitudes associated with each stop
VehLong = []

#marker fill based on pickup or dropoff
VehFill = []

#hovertext RequestID
VehText = []


#create lists of lists
for v in range(30):
    VehRequestID.append([])
    VehLat.append([])
    VehLong.append([])
    VehFill.append([])
    VehText.append([])
    
#calculate request ID associated with each stop
#extract latitude/longitude from requestID
#assign marker fill: pickup = 'circle', dropoff = 'circle-open'

clist = df_Locations.columns
for v in range(30):
    for i, location in enumerate(df_Locations[clist[v]].dropna(),start=1):
        vehdemand =  df_Demand[clist[v]].dropna()[i]
        if vehdemand == 1:
            VehRequestID[v].append(location/2)
            VehLat[v].append(df_Requests.fromLatitude[location/2])
            VehLong[v].append(df_Requests.fromLongitude[location/2])
            VehFill[v].append('circle')
            VehText[v].append('Pickup RequestID '+str(int(location/2)))
        elif vehdemand == -1:
            VehRequestID[v].append((location-1)/2)
            VehLat[v].append(df_Requests.toLatitude[(location-1)/2])
            VehLong[v].append(df_Requests.toLongitude[(location-1)/2])
            VehFill[v].append('circle-open')
            VehText[v].append('Dropoff RequestID '+str(int((location-1)/2)))

            
#define minute by minute interval between first arrival time and last in solution data
interval = range(27010,37388,60)

#create list of positions associated with interval for vehicle
VehPosition = []
            
#minute by minute position for Veh0 all stops
for i in range(len(VehLong[0])-1):
    for z in interval:
        if df_ArrivalTimes[clist[0]].dropna()[i+1] <= z <= df_ArrivalTimes[clist[0]].dropna()[i+2]:
            t = (z - df_ArrivalTimes[clist[0]].dropna()[i+1]) / (df_ArrivalTimes[clist[0]].dropna()[i+2]-df_ArrivalTimes[clist[0]].dropna()[i+1])
            x = VehLong[0][i] + (VehLong[0][i+1]-VehLong[0][i])*t
            y = VehLat[0][i] + (VehLat[0][i+1]-VehLat[0][i])*t
            VehPosition.append((x, y, z))
            
#extract x, y, z coords from positions
positionx = [i[0] for i in VehPosition]
positiony = [i[1] for i in VehPosition]
positionz = [i[2] for i in VehPosition]
    

data = []
for v in range(30):
    trace = go.Scatter3d(
        x=VehLong[v],
        y=VehLat[v],
        z=df_ArrivalTimes[clist[v]].dropna(),
        name='Vehicle' + str(v),
        mode='lines+markers',
        marker=dict(
            size=6,
            symbol=VehFill[v],
            line=dict(
                width=0.5
            ),
            opacity=0.7
        ),
        line=dict(),
        text=VehText[v]
    )
    data.append(trace)

    
#trace to check positions against veh0 route
traceposition = go.Scatter3d(
        x=positionx,
        y=positiony,
        z=positionz,
        name='Veh0 positions',
        mode='markers',
        marker=dict(
            size=3,
            symbol='square',
            line=dict(
                width=0.5
            ),
            opacity=0.7
        ),
        line=dict(),
    )
data.append(traceposition)
    
    
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    ),
    legend=dict(
        x=0,
        y=0,
        traceorder='normal',
        font=dict(
            family='sans-serif',
            size=12,
            color='#000'
        ),
        bgcolor='#E2E2E2',
        bordercolor='#FFFFFF',
        borderwidth=2
    )
)

fig = go.Figure(data=data, layout=layout)
py.offline.iplot(fig)



In [26]:
import plotly.offline as py
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import itertools
from haversine import haversine
from scipy import stats


py.offline.init_notebook_mode(connected=True)

#import data

#datapath = "C:\Users\Dan Steffy\Dropbox\Research\VRPVIS\\"
datapath = "/Users/TomCreo/Desktop/"

df_ArrivalTimes = pd.read_excel(str(datapath+"Assignment30.xlsx"), sheet_name="ArrivalTimes")
df_Locations = pd.read_excel(str(datapath+"Assignment30.xlsx"), sheet_name="Locations")
df_ArrivalLoad = pd.read_excel(str(datapath+"Assignment30.xlsx"), sheet_name="ArrivalLoad")
df_Demand = pd.read_excel(str(datapath+"Assignment30.xlsx"), sheet_name="Demand")
df_Requests = pd.read_csv(str(datapath+"Google_Campus_Data_217_Shifted.csv"))

#to store requestID associated with each stop
VehRequestID = []

#to store latitude associated with each stop
VehLat = []

#to store longitudes associated with each stop
VehLong = []

#marker fill based on pickup or dropoff
VehFill = []

#hovertext RequestID
VehText = []


#create lists of lists
for v in range(30):
    VehRequestID.append([])
    VehLat.append([])
    VehLong.append([])
    VehFill.append([])
    VehText.append([])
    
#calculate request ID associated with each stop
#extract latitude/longitude from requestID
#assign marker fill: pickup = 'circle', dropoff = 'circle-open'

clist = df_Locations.columns
for v in range(30):
    for i, location in enumerate(df_Locations[clist[v]].dropna(),start=1):
        vehdemand =  df_Demand[clist[v]].dropna()[i]
        if vehdemand == 1:
            VehRequestID[v].append(location/2)
            VehLat[v].append(df_Requests.fromLatitude[location/2])
            VehLong[v].append(df_Requests.fromLongitude[location/2])
            VehFill[v].append('circle')
            VehText[v].append('Pickup RequestID '+str(int(location/2)))
        elif vehdemand == -1:
            VehRequestID[v].append((location-1)/2)
            VehLat[v].append(df_Requests.toLatitude[(location-1)/2])
            VehLong[v].append(df_Requests.toLongitude[(location-1)/2])
            VehFill[v].append('circle-open')
            VehText[v].append('Dropoff RequestID '+str(int((location-1)/2)))

            
            
#define minute by minute interval between first arrival time and last in solution data
interval = range(27010,37388,60)

            
#minute by minute position for given vehicle between all stops
def position(VehNum):
    VehPosition = []
    for i in range(len(VehLong[VehNum])-1):
        for z in interval:
            if df_ArrivalTimes[clist[VehNum]].dropna()[i+1] <= z <= df_ArrivalTimes[clist[VehNum]].dropna()[i+2]:
                t = ((z - df_ArrivalTimes[clist[VehNum]].dropna()[i+1]) / 
                        (df_ArrivalTimes[clist[VehNum]].dropna()[i+2]-df_ArrivalTimes[clist[VehNum]].dropna()[i+1])
                    )
                x = VehLong[VehNum][i] + (VehLong[VehNum][i+1]-VehLong[VehNum][i])*t
                y = VehLat[VehNum][i] + (VehLat[VehNum][i+1]-VehLat[VehNum][i])*t
                VehPosition.append((x, y, z))
    return VehPosition;



#calculates avg location for 2 specified vehicles and measures distance between them
#returns distance between avg locations in km

def avg_loc_dist(VehNum1,VehNum2):
    AvgLong1 = sum(VehLong[VehNum1]) / len(VehLong[VehNum1])
    AvgLat1 = sum(VehLat[VehNum1]) / len(VehLat[VehNum1])
    AvgLoc1 = (AvgLong1, AvgLat1)

    AvgLong2 = sum(VehLong[VehNum2]) / len(VehLong[VehNum2])
    AvgLat2 = sum(VehLat[VehNum2]) / len(VehLat[VehNum2])
    AvgLoc2 = (AvgLong2, AvgLat2)
    
    dist = haversine(AvgLoc1, AvgLoc2)
    return dist;


#calculate distance in km for all combinations of 2 vehicles and find minimum overall vehicles
AvgLocDist = []
AvgLocMinDist = None

for pair in itertools.combinations(range(30), 2):
    CurrentDist = avg_loc_dist(*pair)
    AvgLocDist.append([pair, CurrentDist])
    if AvgLocMinDist == None:
        AvgLocMinDist = CurrentDist
    else:
        AvgLocMinDist = min(AvgLocMinDist,CurrentDist)


        
    
    
#calculates distance between 2 specified vehicles at each time t
#returns minimum distance
def min_dist(VehNum1,VehNum2):
    dist2 = []
    VehNum1Pos = position(VehNum1)
    VehNum2Pos = position(VehNum2)
    for t in interval:
        VehNum1Loc_t = [item for item in VehNum1Pos if item[2] == t]
        VehNum2Loc_t = [item for item in VehNum2Pos if item[2] == t]
        if (len(VehNum1Loc_t) != 0 and
            len(VehNum2Loc_t) != 0):
            VehNum1Loc = (VehNum1Loc_t[0][0],VehNum1Loc_t[0][1])
            VehNum2Loc = (VehNum2Loc_t[0][0],VehNum2Loc_t[0][1])
            dist2.append(((haversine(VehNum1Loc, VehNum2Loc))))
    if len(dist2) != 0:
        return min(dist2);
    else:
        return None;
    



#caclulates min distance for all pairs of vehicles
MinLocDist = []

for pair in itertools.combinations(range(30), 2):
    min_distance = min_dist(*pair)
    print [pair, min_distance]
    MinLocDist.append([pair, min_distance])
    

    
    
#calculates avg distance between 2 specified vehicles over time when both are active; returns distance
def avg_dist(VehNum1,VehNum2):
    Distance_t = min_dist(VehNum1,VehNum2)
    DistSum = sum(pair[0] for pair in Distance_t)
    AvgDist = DistSum / len(Distance_t)
    return AvgDist;





#read in experiment output and calculate improvements
df_ExpData = pd.read_excel(str(datapath+"Data 435.xlsx"))

ReoptImprovement = []


for i in range(len(df_ExpData.FirstVehicle)-1):
    ReoptImprovement.append([(int(df_ExpData.FirstVehicle[i+1]), int(df_ExpData.SecondVehicle[i+1])), (int(df_ExpData.OriginalCost_s[i+1]) - int(df_ExpData.MIPTrueCost_s[i+1]))])
    
    



#create 2d scatterplots

ReoptImpY = []
VehPairs = []


for i, data in enumerate(ReoptImprovement):
    if MinLocDist[i][1] != None:
        ReoptImpY.append(data[1])
        VehPairs.append(data[0])

        

#versus MinDist
MinDistX = [i[1] for i in MinLocDist if i[1] != None]

slope1, intercept1, r_value, p_value, std_err = stats.linregress(MinDistX,ReoptImpY)

line1 = slope1*np.array(MinDistX)+intercept1

print r_value

trace0 = go.Scatter(
    x = MinDistX,
    y = ReoptImpY,
    mode = 'markers',
    text = VehPairs,
    name = 'Vehicle Route Pairs'
    )

trace1 = go.Scatter(
    x=MinDistX,
    y=line1,
    mode='lines',
    name='Linear Fit'
    )

data1 = [trace0,trace1]

layout1 = go.Layout(
    title = 'Minimum Distance Between Route Pairs vs. Reoptimized Improvement',
    font=dict(family='Arial, sans-serif', size=12),
    xaxis = dict(
        title = 'Minimum Distance Between Route Pairs Over Time Active (km)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=12
        ),
        tickfont=dict(
            family='Arial, sans-serif',
            size=10
        ),
    ),
    yaxis = dict(
        title = 'Route Cost Improvement upon Reoptimizing Route Pair (s)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=12
        ),
        tickfont=dict(
            family='Arial, sans-serif',
            size=10
        )
    )
)

fig1 = dict(data=data1, layout=layout1)

py.offline.iplot(fig1)

print "R value: "
print r_value


[(0, 1), 0.36088934631176944]
[(0, 2), 0.794281087739417]
[(0, 3), 0.15551061191814927]
[(0, 4), 0.22133795120771851]
[(0, 5), 1.240446166157651]
[(0, 6), 0.805643075274072]
[(0, 7), 0.4635484492165148]
[(0, 8), 0.6612815922780461]
[(0, 9), 0.4219435810919118]
[(0, 10), 0.2366946671259699]
[(0, 11), 0.30680819688443367]
[(0, 12), 1.3402527704641836]
[(0, 13), 0.7306919390821519]
[(0, 14), 0.297968642289175]
[(0, 15), 0.09924052192254608]
[(0, 16), 0.6140307280899803]
[(0, 17), 0.5156616186233366]
[(0, 18), 1.6051059043699698]
[(0, 19), 1.771534379012203]
[(0, 20), 0.5277255598173869]


/Users/TomCreo/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:81: RuntimeWarning:

invalid value encountered in double_scalars



[(0, 21), 0.6316648661361564]
[(0, 22), 0.06117575800713057]
[(0, 23), 1.3196224123038451]
[(0, 24), 2.0600218297062085]
[(0, 25), 0.5892627777806351]
[(0, 26), 0.4561641227069791]
[(0, 27), 0.45668758757792777]
[(0, 28), 0.43515391158803673]
[(0, 29), 0.17374348222392455]
[(1, 2), 0.32247519870640956]
[(1, 3), 0.1099358754225575]
[(1, 4), 0.8318671877108144]
[(1, 5), 0.7020042817410815]
[(1, 6), 0.5816415954500536]
[(1, 7), 0.08494980698780072]
[(1, 8), 0.1928070261895366]
[(1, 9), 0.16653663760641557]
[(1, 10), 0.8784708501513319]
[(1, 11), 0.6492937142441297]
[(1, 12), 1.0758276595950722]
[(1, 13), 0.17240651180016084]
[(1, 14), 0.4808474094894364]
[(1, 15), 1.176526535979739]
[(1, 16), 0.394227283290142]
[(1, 17), 0.06728106669520531]
[(1, 18), 0.7098294274306348]
[(1, 19), 1.41188209962258]
[(1, 20), 1.1789102333912724]
[(1, 21), 0.10348765973527935]
[(1, 22), 0.12633633456911048]
[(1, 23), 0.8321830439681132]
[(1, 24), 0.916730320453768]
[(1, 25), 0.15478538797663555]
[(1, 26), 0

[(12, 23), 0.2508577616611658]
[(12, 24), 0.38942600385488924]
[(12, 25), 0.4944728795560289]
[(12, 26), 0.6238123236821101]
[(12, 27), 1.1440208856544838]
[(12, 28), 1.288188229777912]
[(12, 29), 4.2280490631897525]
[(13, 14), 0.3226205840397256]
[(13, 15), 0.1056921570935145]
[(13, 16), 1.542446695090686]
[(13, 17), 0.6098600924180009]
[(13, 18), 2.467810663553805]
[(13, 19), 0.30404256486202674]
[(13, 20), 2.1710372941704024]
[(13, 21), 0.3673838466251147]
[(13, 22), 0.24964445314071668]
[(13, 23), 2.5421727690509384]
[(13, 24), 0.14708244650535165]
[(13, 25), 0.45147722067270607]
[(13, 26), 2.4542652803322897]
[(13, 27), 0.3427476426063553]
[(13, 28), 2.0016473226453186]
[(13, 29), 0.9016082268730273]
[(14, 15), 0.4913580399795072]
[(14, 16), 0.9580564276963315]
[(14, 17), 0.7957181837667927]
[(14, 18), 0.2809942248872877]
[(14, 19), 1.1979340912937761]
[(14, 20), 0.21334790665387504]
[(14, 21), 2.28579326763584]
[(14, 22), 0.7917233417103212]
[(14, 23), 0.626657782600987]
[(14, 24

R value: 
-0.12036724928353316
